Cryptocurrency trading bot.

Main.

In [1]:
from crypto_logger import init_loggers
crypto_loggers = init_loggers()
crypto_loggers

{'input_15s': <cryptocurrency.crypto_logger_input.Crypto_logger_input at 0x7f7975ec0390>,
 'output_15s': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7f796f963210>,
 'output_1min': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7f7975fa3990>,
 'output_30min': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7f797672b3d0>,
 'output_1h': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7f7976454350>,
 'output_1d': <cryptocurrency.crypto_logger_output.Crypto_logger_output at 0x7f7975edf410>}

In [ ]:
#from crypto_logger import loop_loggers
#loop_loggers(crypto_loggers)

In [5]:
input_15s = crypto_loggers['input_15s']
df_input_15s = input_15s.dataset
df_input_15s

,base_asset,price_change_percent,last_ID,count,rolling_base_volume,bid_volume,ask_volume,close,bid_price,ask_price,bid_ask_percent_change,bid_ask_volume_percent_change,rolling_quote_volume,symbol,quote_asset
date,,,,,,,,,,,,,,,
2022-11-16 18:37:30,IQ,0.000000,10689824.0,871.0,1.305290e+05,0.000000,0.000000,0.0029,0.000000,0.000000,NaN,NaN,1.305290e+05,IQ,IQ
2022-11-16 18:37:45,CREAM,-3.687568,8174435.0,2027.0,3.899360e+05,3507.478904,4956.413496,7.0963,50.100400,50.313300,0.423149,41.440495,3.899360e+05,CREAM,CREAM
2022-11-16 18:38:15,ARK,-3.846154,13016967.0,1726.0,1.676410e+05,4.740000,260.340000,0.2500,0.025473,0.025473,0.000000,1.788139,1.676410e+05,ARK,ARK
2022-11-16 18:39:15,KMD,-1.982851,24500222.0,2609.0,2.122368e+05,3.651500,25.091500,0.1829,0.025659,0.025659,0.000000,12.703963,2.122368e+05,KMD,KMD
2022-11-16 18:39:30,SNT,0.000000,15956776.0,1531.0,1.171393e+05,0.000000,0.000000,0.0200,0.000000,0.000000,NaN,NaN,1.171393e+05,SNT,SNT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-16 18:41:00,GMT,2.572097,202051656.0,178418.0,2.587778e+07,32472.993322,3020.917952,0.3948,0.252669,0.252893,0.088656,91.488912,2.587778e+07,GMT,GMT
2022-11-16 18:41:00,ENJ,-2.448211,151885019.0,25502.0,3.235650e+06,5821.811001,3355.462053,0.3108,0.156485,0.156821,0.214240,63.437265,3.235650e+06,ENJ,ENJ
2022-11-16 18:41:00,UNFI,6.047412,55314131.0,41487.0,4.666659e+06,1989.706800,5541.357850,4.3840,18.697763,18.710627,0.068751,26.419994,4.666659e+06,UNFI,UNFI


In [6]:
output_15s = crypto_loggers['output_15s']
df_output_15s = output_15s.dataset
df_output_15s

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-13 05:19:15,79903.0440,10486.3220,0.000000,0.00000,0.0000,8641.53255,254.7270,0.000000e+00,0.000000e+00,0.0000,...,2.470336e+06,201007.566673,374476.090797,4.987663e+06,3.084278e+06,2.602032e+06,4.597927e+06,1.327384e+06,5.495958e+06,864373.547974
2022-11-13 05:19:30,79903.0440,10486.3220,0.000000,0.00000,0.0000,8641.53255,254.7270,0.000000e+00,0.000000e+00,0.0000,...,2.470336e+06,201007.566673,374476.090797,4.987663e+06,3.084278e+06,2.602032e+06,4.597927e+06,1.327384e+06,5.495958e+06,864373.547974
2022-11-13 05:19:45,79903.0440,10486.3220,0.000000,0.00000,0.0000,8641.53255,254.7270,0.000000e+00,0.000000e+00,0.0000,...,2.470336e+06,201007.566673,374476.090797,4.987663e+06,3.084278e+06,2.602032e+06,4.597927e+06,1.327384e+06,5.495958e+06,864373.547974
2022-11-13 05:20:00,3541.7580,698.2736,320.604349,10.10355,0.0000,15307.11570,1098.8370,1.517767e-08,0.000000e+00,60.1344,...,2.465459e+06,200987.567973,374414.946031,4.992638e+06,3.084676e+06,2.601419e+06,4.587839e+06,1.325331e+06,5.490953e+06,864425.580874
2022-11-13 05:20:15,3541.7580,698.2736,320.604349,10.10355,0.0000,15307.11570,1098.8370,1.517767e-08,0.000000e+00,60.1344,...,2.465459e+06,200987.567973,374414.946031,4.992638e+06,3.084676e+06,2.601419e+06,4.587839e+06,1.325331e+06,5.490953e+06,864425.580874
2022-11-13 05:20:30,3541.7580,698.2736,320.604349,10.10355,0.0000,15307.11570,1098.8370,1.517767e-08,0.000000e+00,60.1344,...,2.465459e+06,200987.567973,374414.946031,4.992638e+06,3.084676e+06,2.601419e+06,4.587839e+06,1.325331e+06,5.490953e+06,864425.580874
2022-11-13 05:20:45,3541.7580,698.2736,320.604349,10.10355,0.0000,15307.11570,1098.8370,1.517767e-08,0.000000e+00,60.1344,...,2.465459e+06,200987.567973,374414.946031,4.992638e+06,3.084676e+06,2.601419e+06,4.587839e+06,1.325331e+06,5.490953e+06,864425.580874
2022-11-13 05:21:00,339.1960,2702.9660,0.000000,0.00000,0.0000,14250.06980,925.2840,0.000000e+00,1.836097e-06,335.8350,...,2.465170e+06,200442.708627,373828.041096,4.992957e+06,3.085553e+06,2.603664e+06,4.559356e+06,1.325398e+06,5.488959e+06,864425.580874


In [7]:
output_1min = crypto_loggers['output_1min']
df_output_1min = output_1min.dataset
df_output_1min

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-12 04:35:00,1.033589e+04,2.653060e+04,9.693936e+01,1.631001e+03,4.619732e+02,82101.46275,4.941955e+03,7.530817e-06,3.488950e-07,245.6681,...,5.780858e+06,444304.698564,1.263348e+06,4.528592e+06,7.270292e+06,1.068880e+07,6.321041e+06,2.142217e+06,1.038329e+07,2.135383e+06
2022-11-12 04:36:00,3.588140e+03,2.096250e+04,1.024951e+02,1.011998e+02,0.000000e+00,187409.30040,3.167380e+03,1.066272e-06,2.340318e-08,1020.5539,...,5.805909e+06,444321.640364,1.263384e+06,4.543262e+06,7.270339e+06,1.068890e+07,6.337747e+06,2.143049e+06,1.038481e+07,2.137903e+06
2022-11-12 04:37:00,6.994926e+03,3.781936e+04,1.111225e+01,2.999133e+01,1.383626e+03,78340.95701,1.932476e+03,3.574382e-08,0.000000e+00,456.8963,...,5.819726e+06,444321.640364,1.263384e+06,4.556204e+06,7.272130e+06,1.069388e+07,6.343576e+06,2.143125e+06,1.038719e+07,2.138139e+06
2022-11-12 04:38:00,4.804915e+03,2.570429e+04,3.667305e+01,7.160172e+01,3.247372e+03,37950.20600,5.285904e+03,0.000000e+00,5.373915e-07,0.0000,...,5.829789e+06,444321.640364,1.263857e+06,4.560548e+06,7.272404e+06,1.069407e+07,6.343576e+06,2.143147e+06,1.039078e+07,2.139021e+06
2022-11-12 04:39:00,7.267722e+03,4.812196e+04,0.000000e+00,1.964000e+01,2.858948e+03,59290.49273,3.436742e+03,0.000000e+00,5.907149e-07,0.0000,...,5.831748e+06,444321.640364,1.265060e+06,4.563169e+06,7.272432e+06,1.069409e+07,6.344267e+06,2.143353e+06,1.039569e+07,2.139238e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-13 05:30:00,9.761922e+02,5.189173e+03,0.000000e+00,0.000000e+00,0.000000e+00,29873.37942,1.144480e+01,0.000000e+00,0.000000e+00,0.0000,...,2.430718e+06,198957.785312,3.743769e+05,4.944264e+06,3.093880e+06,2.578602e+06,4.623702e+06,1.315944e+06,5.437307e+06,8.598834e+05
2022-11-13 05:31:00,8.104572e+02,1.829520e+01,0.000000e+00,5.857553e+02,0.000000e+00,45902.21677,1.689237e+02,0.000000e+00,1.022903e-05,0.0000,...,2.424537e+06,198851.676302,3.736898e+05,4.935831e+06,3.093421e+06,2.571392e+06,4.616149e+06,1.314023e+06,5.429730e+06,8.588905e+05


In [8]:
output_30min = crypto_loggers['output_30min']
df_output_30min = output_30min.dataset
df_output_30min

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-12 00:00:00,799.1944,7305.8328,587.393826,209.12000,138.8832,82127.34230,2945.3508,3.182141e-08,4.025402e-07,16.0146,...,5.073841e+06,400765.757514,1.177818e+06,3.841247e+06,6.641584e+06,9.967398e+06,5.782797e+06,1.909429e+06,9.239983e+06,1.861429e+06
2022-11-12 00:30:00,1117.9448,4439.4668,0.000000,0.00000,0.0000,19112.38462,1156.5396,4.034262e-05,1.451355e-07,73.2000,...,5.156743e+06,402475.175255,1.190377e+06,3.881614e+06,6.769918e+06,1.007757e+07,5.817664e+06,1.927583e+06,9.367546e+06,1.877043e+06
2022-11-12 01:00:00,1901.8888,7368.9590,20.002022,1903.93742,1.7353,42863.50656,2247.8720,1.453814e-06,0.000000e+00,578.6802,...,5.212918e+06,404253.904201,1.196600e+06,3.904480e+06,6.796906e+06,1.028341e+07,5.834727e+06,1.936683e+06,9.456096e+06,1.908278e+06
2022-11-12 01:30:00,31.0750,1049.7861,345.659882,33.62881,614.4285,30031.33578,1855.0308,6.744523e-07,2.228348e-07,488.8188,...,5.332883e+06,406073.710920,1.198351e+06,3.920506e+06,6.847694e+06,1.032332e+07,5.872226e+06,1.947277e+06,9.563474e+06,1.920464e+06
2022-11-12 02:00:00,506.5500,8017.8630,440.410560,148.31000,38.0304,45386.84010,35.2655,2.794634e-06,1.125199e-06,0.0000,...,5.366630e+06,407990.668646,1.198790e+06,3.940283e+06,6.972526e+06,1.036621e+07,5.906028e+06,1.960785e+06,9.676614e+06,1.979933e+06
2022-11-12 02:30:00,437.2185,1988.6328,0.000000,359.50950,0.0000,27922.75002,1714.9600,0.000000e+00,6.114364e-07,244.6041,...,5.395612e+06,421235.737404,1.205423e+06,3.946190e+06,7.072812e+06,1.046953e+07,5.931026e+06,1.974640e+06,9.779096e+06,2.016835e+06
2022-11-12 03:00:00,2939.6327,24254.7636,36.720960,16.79328,0.0000,155372.52718,1160.3792,0.000000e+00,3.541070e-07,0.0000,...,5.446186e+06,421970.529844,1.218331e+06,3.976806e+06,7.110819e+06,1.054173e+07,5.946172e+06,1.989712e+06,9.905847e+06,2.051310e+06
2022-11-12 03:30:00,19.0903,8150.1222,0.000000,0.00000,0.0000,19573.57710,6406.1475,5.107331e-08,3.454221e-07,18.1300,...,5.567339e+06,434875.165840,1.228123e+06,4.019565e+06,7.160568e+06,1.059612e+07,6.041151e+06,2.025071e+06,1.000896e+07,2.065041e+06


In [9]:
output_1h = crypto_loggers['output_1h']
df_output_1h = output_1h.dataset
df_output_1h

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-10 17:00:00,4.569324e+05,1.740717e+06,47260.590023,1.097438e+05,69168.8454,8.921183e+06,191443.2394,0.000059,0.000238,50470.9554,...,1.804428e+05,62309.596952,3.377431e+04,4.756686e+05,2.791954e+05,2.365043e+05,4.520270e+05,1.401489e+05,8.375013e+05,9.609081e+04
2022-11-10 18:00:00,1.006092e+05,9.437831e+05,37309.029088,3.245254e+05,84922.2414,4.936650e+06,180465.1020,0.000085,0.000129,14082.1632,...,2.762510e+05,113558.341771,5.141262e+04,5.835107e+05,4.378708e+05,2.950974e+05,7.817690e+05,2.196924e+05,1.072005e+06,2.076141e+05
2022-11-10 19:00:00,1.005070e+05,6.226492e+05,11835.661337,3.776497e+04,26220.4896,3.946059e+06,174980.8851,0.000055,0.000113,12622.4792,...,4.060516e+05,145847.197535,6.249334e+04,8.862145e+05,4.889899e+05,3.759254e+05,9.841157e+05,2.990276e+05,1.606705e+06,3.307760e+05
2022-11-10 20:00:00,2.782391e+05,9.027705e+05,21849.158279,6.139533e+04,32739.2580,7.054816e+06,200208.8892,0.000042,0.000138,16133.9203,...,6.773062e+05,184655.535836,9.251780e+04,1.077321e+06,7.025528e+05,5.098590e+05,1.185612e+06,4.070237e+05,2.024846e+06,4.740459e+05
2022-11-10 21:00:00,1.930421e+05,9.376701e+05,20622.593442,4.860193e+04,43511.7176,7.408349e+06,161832.9460,0.000083,0.000067,17723.2326,...,8.726880e+05,204700.659764,1.314218e+05,1.598601e+06,9.124114e+05,6.375344e+05,1.678826e+06,5.381757e+05,2.189689e+06,5.760737e+05
2022-11-10 22:00:00,1.196164e+05,4.239776e+05,9477.649278,2.083570e+04,12726.4818,3.418581e+06,883819.1892,0.000054,0.000076,19026.9840,...,1.076374e+06,215185.446942,1.462513e+05,1.705435e+06,1.034302e+06,7.007605e+05,1.989828e+06,5.757013e+05,2.392910e+06,6.225255e+05
2022-11-10 23:00:00,4.534314e+05,5.855634e+05,7546.084308,6.450081e+04,22289.5082,4.453425e+06,109263.9612,0.000082,0.000103,28627.8982,...,1.203139e+06,230434.672375,2.109465e+05,1.866188e+06,1.142283e+06,8.887894e+05,2.202196e+06,6.678801e+05,3.584591e+06,7.483302e+05
2022-11-11 00:00:00,3.361588e+05,6.902292e+05,9443.308920,5.971684e+04,49744.1623,6.572796e+06,87729.7626,0.000277,0.000072,41101.3632,...,1.430502e+06,388479.023892,2.703907e+05,2.116801e+06,1.212570e+06,1.097851e+06,2.353599e+06,8.750321e+05,4.056653e+06,9.214655e+05


In [10]:
output_1d = crypto_loggers['output_1d']
df_output_1d = output_1d.dataset
df_output_1d

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-07,6.315383e+06,2.257186e+07,7.438017e+05,3.539144e+06,1.926436e+06,9.392098e+07,4.433607e+06,0.001190,0.019514,2.637851e+06,...,1.102128e+07,1.846253e+06,1.752387e+06,6.944641e+06,2.325496e+06,1.636929e+07,1.129265e+07,3.206494e+06,2.905358e+07,3.455433e+06
2022-11-08,1.402739e+07,5.792126e+07,2.202770e+06,3.252198e+06,4.666886e+06,2.463825e+08,5.959429e+06,0.003021,0.011557,4.522595e+06,...,1.334788e+07,1.589880e+06,4.916578e+06,1.292803e+07,1.593125e+07,1.683744e+07,1.540258e+07,8.102547e+06,2.598021e+07,6.353783e+06
2022-11-09,1.299763e+07,3.568309e+07,2.273469e+06,3.908723e+06,2.022032e+06,2.189370e+08,5.265419e+06,0.004063,0.008319,1.999104e+06,...,1.350249e+07,3.651628e+06,2.969245e+06,1.038509e+07,2.471788e+07,9.173132e+06,1.372610e+07,4.982295e+06,2.053381e+07,4.768127e+06
2022-11-10,1.699758e+07,4.060239e+07,9.192837e+05,3.653941e+06,1.815948e+06,2.446289e+08,5.748716e+06,0.002946,0.004983,1.729121e+06,...,6.911956e+06,1.011332e+06,1.876456e+06,9.288858e+06,7.414404e+06,8.135913e+06,1.068100e+07,3.989579e+06,1.897900e+07,3.918952e+06
2022-11-11,7.080629e+06,2.313415e+07,7.363756e+05,1.961386e+06,1.274227e+06,1.178729e+08,3.825939e+06,0.002079,0.003448,1.895906e+06,...,5.975146e+06,6.500135e+05,1.429405e+06,4.514353e+06,6.990514e+06,1.127158e+07,7.205504e+06,2.685587e+06,1.132494e+07,2.266519e+06
2022-11-12,6.322697e+06,3.712175e+07,3.738035e+05,1.585204e+06,1.052912e+06,7.259718e+07,2.736141e+06,0.001325,0.001414,9.684306e+05,...,3.194772e+06,2.418563e+05,4.350593e+05,5.478773e+06,3.475172e+06,3.089467e+06,5.201255e+06,1.346108e+06,6.080444e+06,1.073600e+06
2022-11-13,8.056914e+05,4.541453e+06,1.416102e+05,7.261847e+04,4.261439e+04,9.623929e+06,2.463911e+05,0.000223,0.000311,1.322700e+05,...,3.475809e+05,1.641643e+04,5.087427e+04,8.211473e+05,3.738122e+05,4.257405e+05,2.207664e+05,1.821211e+05,8.474589e+05,1.536556e+05


In [11]:
df = df_output_1d.copy()
df

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-07,6.315383e+06,2.257186e+07,7.438017e+05,3.539144e+06,1.926436e+06,9.392098e+07,4.433607e+06,0.001190,0.019514,2.637851e+06,...,1.102128e+07,1.846253e+06,1.752387e+06,6.944641e+06,2.325496e+06,1.636929e+07,1.129265e+07,3.206494e+06,2.905358e+07,3.455433e+06
2022-11-08,1.402739e+07,5.792126e+07,2.202770e+06,3.252198e+06,4.666886e+06,2.463825e+08,5.959429e+06,0.003021,0.011557,4.522595e+06,...,1.334788e+07,1.589880e+06,4.916578e+06,1.292803e+07,1.593125e+07,1.683744e+07,1.540258e+07,8.102547e+06,2.598021e+07,6.353783e+06
2022-11-09,1.299763e+07,3.568309e+07,2.273469e+06,3.908723e+06,2.022032e+06,2.189370e+08,5.265419e+06,0.004063,0.008319,1.999104e+06,...,1.350249e+07,3.651628e+06,2.969245e+06,1.038509e+07,2.471788e+07,9.173132e+06,1.372610e+07,4.982295e+06,2.053381e+07,4.768127e+06
2022-11-10,1.699758e+07,4.060239e+07,9.192837e+05,3.653941e+06,1.815948e+06,2.446289e+08,5.748716e+06,0.002946,0.004983,1.729121e+06,...,6.911956e+06,1.011332e+06,1.876456e+06,9.288858e+06,7.414404e+06,8.135913e+06,1.068100e+07,3.989579e+06,1.897900e+07,3.918952e+06
2022-11-11,7.080629e+06,2.313415e+07,7.363756e+05,1.961386e+06,1.274227e+06,1.178729e+08,3.825939e+06,0.002079,0.003448,1.895906e+06,...,5.975146e+06,6.500135e+05,1.429405e+06,4.514353e+06,6.990514e+06,1.127158e+07,7.205504e+06,2.685587e+06,1.132494e+07,2.266519e+06
2022-11-12,6.322697e+06,3.712175e+07,3.738035e+05,1.585204e+06,1.052912e+06,7.259718e+07,2.736141e+06,0.001325,0.001414,9.684306e+05,...,3.194772e+06,2.418563e+05,4.350593e+05,5.478773e+06,3.475172e+06,3.089467e+06,5.201255e+06,1.346108e+06,6.080444e+06,1.073600e+06
2022-11-13,8.056914e+05,4.541453e+06,1.416102e+05,7.261847e+04,4.261439e+04,9.623929e+06,2.463911e+05,0.000223,0.000311,1.322700e+05,...,3.475809e+05,1.641643e+04,5.087427e+04,8.211473e+05,3.738122e+05,4.257405e+05,2.207664e+05,1.821211e+05,8.474589e+05,1.536556e+05


In [13]:
df['BTC']

pair,base_volume,close,high,low,open,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-11-07,1.199179e+10,20591.13,21069.77,20384.89,20905.58,1.208156e+10,1.922697e+10,1.940456e+10
2022-11-08,2.273489e+10,18547.23,20700.88,17166.83,20590.67,2.364546e+10,2.273489e+10,2.364546e+10
2022-11-09,1.786324e+10,15922.81,18587.76,15588.00,18545.38,1.937955e+10,1.786324e+10,1.937955e+10
2022-11-10,1.732047e+10,17601.15,18199.00,15754.26,15922.68,1.677071e+10,1.732047e+10,1.677071e+10
2022-11-11,1.058466e+10,17070.31,17695.00,16361.60,17602.45,1.060168e+10,1.058466e+10,1.060168e+10
2022-11-12,4.540573e+09,16812.08,17119.10,16631.39,17069.98,4.546249e+09,4.540573e+09,4.546249e+09
2022-11-13,6.072625e+08,16883.05,16954.28,16774.07,16813.16,6.075869e+08,6.072625e+08,6.075869e+08


In [ ]:
import pandas_ta as ta

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_relative_volume_levels_smoothed_trigger(data, average1=26, average2=14):
    volume = data['volume']
    volume_average = ta.sma(close=volume, length=average1)
    relative_volume = volume / volume_average
    smoothed_relative_volume = ta.sma(close=relative_volume, length=average2)
    return (smoothed_relative_volume > threshold).iat[-1]

def get_relative_volume_levels_at_time_smoothed_thresholded(data):
    try:
        volume = data['volume']
        #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
        cum_volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
        #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
        cum_rvol = (cum_volume / cum_volume.shift(1)).fillna(method='pad')
        rvol = (volume / volume.shift(1)).fillna(method='pad')
        bar_up = (data['close'] > data['open'])
        bar_up |= (data['close'] == data['open']) & (data['close'].diff() > 0)
        bar_up = bar_up.astype(int)
        bar_up = bar_up * 2 - 1
        #rvol *= bar_up
        cum_rvol_dir = cum_rvol * bar_up
        rvol_dir = rvol * bar_up
        rvol_indicator = ta.hma(rvol, length=14, talib=True)
        rvol_dir_indicator = ta.hma(rvol_dir, length=14, talib=True)
        cum_rvol_indicator = ta.hma(cum_rvol, length=14, talib=True)
        cum_rvol_dir_indicator = ta.hma(cum_rvol_dir, length=14, talib=True)
        rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
        rvol_dir_indicator = rvol_dir_indicator.rename('relative_volume_levels_dir_smoothed')
        cum_rvol_indicator = cum_rvol_indicator.rename('cum_relative_volume_levels_smoothed')
        cum_rvol_dir_indicator = cum_rvol_dir_indicator.rename('cum_relative_volume_levels_dir_smoothed')
        #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
        threshold_dir = 0
        threshold = 2
        rvol_thresholded = (rvol_indicator > threshold).iat[-1]
        rvol_dir_thresholded = (rvol_dir_indicator > threshold_dir).iat[-1]
        cum_rvol_thresholded = (cum_rvol_indicator > threshold).iat[-1]
        cum_rvol_dir_thresholded = (cum_rvol_dir_indicator > threshold_dir).iat[-1]
        trigger = (rvol_thresholded | rvol_dir_thresholded | cum_rvol_thresholded | cum_rvol_dir_thresholded)
    except Exception as e:
        print('rvol exception:', e)
        trigger = False
    return trigger

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iat[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iat[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iat[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iat[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iat[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iat[-2]) or \
            (MACD['MACD_12_26_9'].iat[-1] > MACD['MACDs_12_26_9'].iat[-1]))

def get_not_square_wave_triggers(data, multiplier_schedule):
    triggers = True
    for multiplier in multiplier_schedule:
        period_1 = -4 * multiplier
        uniques_1 = 2 * multiplier
        square_wave_trigger_1 = (data.iloc[period_1:]['close'].unique().size < uniques_1)
        if square_wave_trigger_1:
            triggers = False
            break
        else:
            period_2 = -15 * multiplier
            uniques_2 = 6 * multiplier
            square_wave_trigger_2 = (data.iloc[period_2:]['close'].unique().size < uniques_2)
            if square_wave_trigger_2:
                triggers = False
                break
    return triggers

def get_minute_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 5, 10, 15, 20, 45])

def get_hourly_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_volume_minimum_trigger(data):
    return (data['volume'] > 1000000).iat[-1]

def get_daily_volume_change_trigger(data):
    return ((data['volume'].pct_change(1) * 100) > 300).iat[-1]

def get_minute_daily_volume_minimum_trigger(data):
    return (data['rolling_base_volume'] > 1000000).iat[-1]

def get_minute_daily_volume_change_trigger(data):
    return ((data['rolling_base_volume'].pct_change(1440) * 100) > 300).iat[-1]

def get_rising_volume_trigger(data):
    return (data['rolling_base_volume'].diff(1) > 0).iat[-1]

In [ ]:
df = df_1d.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_daily_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_relative_volume_levels_smoothed_trigger, df).tolist())
filtered_1d = pd.Series(list(filtered_1 & filtered_2 & filtered_3))
filtered_1d

In [ ]:
df = df_1h.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_hourly_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_relative_volume_levels_at_time_smoothed_thresholded, df).tolist())
filtered_1h = pd.Series(list(filtered_1 & filtered_2))
filtered_1h

In [ ]:
from cryptocurrency.indicators import filter_in_market, screen_one
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered = filter_in_market(screen_one, df).tolist()
filtered

In [ ]:
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_minute_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_minute_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_minute_daily_volume_change_trigger, df).tolist())
filtered_4 = set(filter_in_market(get_rising_volume_trigger, df).tolist())
filtered_1min = pd.Series(list(filtered_1 & filtered_2 & filtered_3 & filtered_4))
filtered_1min

In [ ]:
list(set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table

offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=False)
conversion_table.sort_values(by='rolling_base_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
conversion_table['quote_asset'].unique()

In [ ]:
def get_connected_assets(asset, exchange_info, priority='accuracy'):
    def reorder(connected_assets, priority):
        prioritized = [asset for asset in priority if asset in connected_assets]
        order = {asset: i for i, asset in enumerate(prioritized)}
        connected_assets_items = [asset for asset in connected_assets if asset in order]
        connected_assets_items.sort(key=order.get)
        connected_assets_iter = iter(connected_assets_items)
        return [next(connected_assets_iter) if asset in order 
                else asset for asset in connected_assets]
    priorities = {}
    priorities['accuracy'] = ['USDT', 'BTC', 'BUSD', 'ETH', 'BNB', 'AUD']
    priorities['fees'] = ['BUSD', 'BTC', 'BNB', 'ETH', 'USDT', 'AUD']
    priorities['wallet'] = ['BTC', 'ETH', 'BUSD', 'BNB', 'USDT', 'AUD']
    priority = priorities[priority]
    connected_base_assets = exchange_info['base_asset'] == asset
    connected_base_assets = exchange_info[connected_base_assets]
    connected_base_assets = connected_base_assets['quote_asset'].tolist()
    connected_quote_assets = exchange_info['quote_asset'] == asset
    connected_quote_assets = exchange_info[connected_quote_assets]
    connected_quote_assets = connected_quote_assets['base_asset'].tolist()
    connected_assets = sorted(list(set(connected_base_assets + connected_quote_assets)))
    prioritized = reorder(connected_assets, priority)
    return prioritized

asset = 'AXS'
connected_assets = get_connected_assets(asset, exchange_info, priority='fees')
print(connected_assets)

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

In [ ]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

In [ ]:
new_conversion_table['ETH']